# Exercise List 6

Utilizando o Apache Spark e demais ferramentas correlatas, além do grafo construído na lista 5, implemente as seguintes operações:

1. Determine o nó central através do grau.
2. Determine o nó central por centralidade utilizando a distância de Wasserman e a distância harmônica (consultar material).
3. Determine o nó central por intermediação.

**Setup**

In [1]:
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

conf = SparkConf().setAppName('appName').setMaster('local')
sc = SparkContext.getOrCreate(conf)
spark = SparkSession(sc)

### Exercise 5

**Crie um CSV para armazenar as cidades, com: `id` (nome da cidade), `latitude`, `longitude` e `população`**

In [2]:
cidades = spark.read.format("csv").option("header", "true").load("data/transport/transport-nodes.csv")
cidades.show()

+---------+--------+---------+----------+
|       id|latitude|longitude|population|
+---------+--------+---------+----------+
|Araripina|  7.5766|  40.4976|     84418|
|  Caruaru|  8.2760|  35.9819|    277982|
| Igarassu|   78292|   349016|     91953|
|  Cabrobo|  8.5082|  39.3103|     33856|
|  Carpina|  7.8450|  35.2437|     81054|
| Ouricuri|  7.8809|  40.0810|     69459|
|  Surubim|  7.8543|  35.7630|     64520|
|Petrolina|  9.3891|  40.5031|       217|
|Salgueiro|  8.0725|  39.1268|     59769|
|   Recife|  8.0522|  34.9286|   1555000|
+---------+--------+---------+----------+



**Crie outro CSV para armazenar a distância entre essas cidades, com: `src`, `dst` e `relationship` como campos**

In [3]:
distancias = spark.read.format("csv").option("header", "true").load("data/transport/transport-relationships.csv")
distancias.show(30)

+---------+---------+------------+----+
|      src|      dst|relationship|cost|
+---------+---------+------------+----+
|Araripina|  Caruaru|       EROAD| 551|
|Araripina| Igarassu|       EROAD| 700|
|Araripina|  Cabrobo|       EROAD| 186|
|   Recife|  Surubim|       EROAD| 118|
|   Recife|Salgueiro|       EROAD| 510|
|   Recife|  Caruaru|       EROAD| 510|
| Ouricuri| Igarassu|       EROAD| 641|
| Ouricuri|  Surubim|       EROAD| 525|
| Ouricuri|Salgueiro|       EROAD| 359|
|  Cabrobo| Ouricuri|       EROAD| 127|
|  Cabrobo|  Carpina|       EROAD| 502|
|  Cabrobo|   Recife|       EROAD| 526|
| Igarassu|Salgueiro|       EROAD| 529|
| Igarassu| Ouricuri|       EROAD| 197|
| Igarassu|   Recife|       EROAD|  28|
|  Carpina|Petrolina|       EROAD| 681|
|  Carpina|Salgueiro|       EROAD| 480|
|  Carpina| Ouricuri|       EROAD| 591|
|Petrolina| Ouricuri|       EROAD| 154|
|Petrolina|  Surubim|       EROAD|  66|
|Petrolina| Igarassu|       EROAD| 726|
|  Surubim|Araripina|       EROAD| 583|


### 2. Utilizando as bibliotecas do Spark, crie um objeto GraphFrame a partir desses dois CSVs.

In [4]:
from graphframes import *

g = GraphFrame(cidades, distancias)

In [5]:
g.vertices.show()

+---------+--------+---------+----------+
|       id|latitude|longitude|population|
+---------+--------+---------+----------+
|Araripina|  7.5766|  40.4976|     84418|
|  Caruaru|  8.2760|  35.9819|    277982|
| Igarassu|   78292|   349016|     91953|
|  Cabrobo|  8.5082|  39.3103|     33856|
|  Carpina|  7.8450|  35.2437|     81054|
| Ouricuri|  7.8809|  40.0810|     69459|
|  Surubim|  7.8543|  35.7630|     64520|
|Petrolina|  9.3891|  40.5031|       217|
|Salgueiro|  8.0725|  39.1268|     59769|
|   Recife|  8.0522|  34.9286|   1555000|
+---------+--------+---------+----------+



In [6]:
g.edges.show()

+---------+---------+------------+----+
|      src|      dst|relationship|cost|
+---------+---------+------------+----+
|Araripina|  Caruaru|       EROAD| 551|
|Araripina| Igarassu|       EROAD| 700|
|Araripina|  Cabrobo|       EROAD| 186|
|   Recife|  Surubim|       EROAD| 118|
|   Recife|Salgueiro|       EROAD| 510|
|   Recife|  Caruaru|       EROAD| 510|
| Ouricuri| Igarassu|       EROAD| 641|
| Ouricuri|  Surubim|       EROAD| 525|
| Ouricuri|Salgueiro|       EROAD| 359|
|  Cabrobo| Ouricuri|       EROAD| 127|
|  Cabrobo|  Carpina|       EROAD| 502|
|  Cabrobo|   Recife|       EROAD| 526|
| Igarassu|Salgueiro|       EROAD| 529|
| Igarassu| Ouricuri|       EROAD| 197|
| Igarassu|   Recife|       EROAD|  28|
|  Carpina|Petrolina|       EROAD| 681|
|  Carpina|Salgueiro|       EROAD| 480|
|  Carpina| Ouricuri|       EROAD| 591|
|Petrolina| Ouricuri|       EROAD| 154|
|Petrolina|  Surubim|       EROAD|  66|
+---------+---------+------------+----+
only showing top 20 rows



### 3. Utilizando o método `bfs` (Breadth First Search), execute **5** filtragens a sua escolha.

**Decent and updated [graphframes documentation](https://docs.databricks.com/_static/notebooks/graphframes-user-guide-py.html)**

In [7]:
g.bfs(fromExpr = "id = 'Araripina'",
      toExpr = "id = 'Recife'",
      edgeFilter = "relationship = 'EROAD'").show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                from|                  e0|                  v1|                  e1|                  to|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|[Araripina, 7.576...|[Araripina, Igara...|[Igarassu, 78292,...|[Igarassu, Recife...|[Recife, 8.0522, ...|
|[Araripina, 7.576...|[Araripina, Cabro...|[Cabrobo, 8.5082,...|[Cabrobo, Recife,...|[Recife, 8.0522, ...|
+--------------------+--------------------+--------------------+--------------------+--------------------+



In [8]:
g.bfs(fromExpr = "id = 'Surubim'",
      toExpr = "id = 'Recife'",
      edgeFilter = "relationship = 'EROAD'").show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                from|                  e0|                  v1|                  e1|                  v2|                  e2|                  to|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|[Surubim, 7.8543,...|[Surubim, Araripi...|[Araripina, 7.576...|[Araripina, Igara...|[Igarassu, 78292,...|[Igarassu, Recife...|[Recife, 8.0522, ...|
|[Surubim, 7.8543,...|[Surubim, Caruaru...|[Caruaru, 8.2760,...|[Caruaru, Igarass...|[Igarassu, 78292,...|[Igarassu, Recife...|[Recife, 8.0522, ...|
|[Surubim, 7.8543,...|[Surubim, Araripi...|[Araripina, 7.576...|[Araripina, Cabro...|[Cabrobo, 8.5082,...|[Cabrobo, Recife,...|[Recife, 8.0522, ...|
|[Surubim, 7.8543,...|[Surubim, Salguei...|[Salgueiro, 8.072...|[Salgueiro, Cabro...|[Cabrobo, 8.5082,...|

In [9]:
g.bfs(fromExpr = "id = 'Salgueiro'",
      toExpr = "id = 'Recife'",
      edgeFilter = "relationship = 'EROAD'").show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                from|                  e0|                  v1|                  e1|                  to|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|[Salgueiro, 8.072...|[Salgueiro, Cabro...|[Cabrobo, 8.5082,...|[Cabrobo, Recife,...|[Recife, 8.0522, ...|
+--------------------+--------------------+--------------------+--------------------+--------------------+



In [10]:
g.bfs(fromExpr = "id = 'Carpina'",
      toExpr = "id = 'Recife'",
      edgeFilter = "relationship = 'EROAD'").show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                from|                  e0|                  v1|                  e1|                  v2|                  e2|                  to|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|[Carpina, 7.8450,...|[Carpina, Petroli...|[Petrolina, 9.389...|[Petrolina, Igara...|[Igarassu, 78292,...|[Igarassu, Recife...|[Recife, 8.0522, ...|
|[Carpina, 7.8450,...|[Carpina, Ouricur...|[Ouricuri, 7.8809...|[Ouricuri, Igaras...|[Igarassu, 78292,...|[Igarassu, Recife...|[Recife, 8.0522, ...|
|[Carpina, 7.8450,...|[Carpina, Salguei...|[Salgueiro, 8.072...|[Salgueiro, Cabro...|[Cabrobo, 8.5082,...|[Cabrobo, Recife,...|[Recife, 8.0522, ...|
+--------------------+--------------------+--------------------+--------------------+--------------------+

In [11]:
g.bfs(fromExpr = "id = 'Cabrobo'",
      toExpr = "id = 'Recife'",
      edgeFilter = "relationship = 'EROAD'").show()

+--------------------+--------------------+--------------------+
|                from|                  e0|                  to|
+--------------------+--------------------+--------------------+
|[Cabrobo, 8.5082,...|[Cabrobo, Recife,...|[Recife, 8.0522, ...|
+--------------------+--------------------+--------------------+



In [12]:
g.bfs(fromExpr = "id = 'Igarassu'",
      toExpr = "id = 'Recife'",
      edgeFilter = "relationship = 'EROAD'").show()

+--------------------+--------------------+--------------------+
|                from|                  e0|                  to|
+--------------------+--------------------+--------------------+
|[Igarassu, 78292,...|[Igarassu, Recife...|[Recife, 8.0522, ...|
+--------------------+--------------------+--------------------+



### 4. Execute 2 consultas utilizando o método `find`.

In [13]:
# find vertices with edges in both directions between them
g.find("(a)-[e]->(b); (b)-[e2]->(a)").show()

+--------------------+--------------------+--------------------+--------------------+
|                   a|                   e|                   b|                  e2|
+--------------------+--------------------+--------------------+--------------------+
|[Ouricuri, 7.8809...|[Ouricuri, Igaras...|[Igarassu, 78292,...|[Igarassu, Ouricu...|
|[Igarassu, 78292,...|[Igarassu, Ouricu...|[Ouricuri, 7.8809...|[Ouricuri, Igaras...|
+--------------------+--------------------+--------------------+--------------------+



In [14]:
# find vertices in chains of three
g.find("(a)-[ab]->(b); (b)-[bc]->(c)").show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                   a|                  ab|                   b|                  bc|                   c|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|[Araripina, 7.576...|[Araripina, Carua...|[Caruaru, 8.2760,...|[Caruaru, Ouricur...|[Ouricuri, 7.8809...|
|[Araripina, 7.576...|[Araripina, Carua...|[Caruaru, 8.2760,...|[Caruaru, Petroli...|[Petrolina, 9.389...|
|[Araripina, 7.576...|[Araripina, Carua...|[Caruaru, 8.2760,...|[Caruaru, Igarass...|[Igarassu, 78292,...|
|[Araripina, 7.576...|[Araripina, Igara...|[Igarassu, 78292,...|[Igarassu, Recife...|[Recife, 8.0522, ...|
|[Araripina, 7.576...|[Araripina, Igara...|[Igarassu, 78292,...|[Igarassu, Ouricu...|[Ouricuri, 7.8809...|
|[Araripina, 7.576...|[Araripina, Igara...|[Igarassu, 78292,...|[Igarassu, Salgue...|[Salgueiro, 8.072...|
|[Araripina, 7.576...|[Araripina, Cab

### 5. Execute 2 consultas utilizando o método `filterVertices`.

**Filter 1: g2**

In [15]:
g2 = g.filterEdges("cost < 500").filterVertices("population > 100000 or latitude < 8")

In [16]:
g2.vertices.show()

+---------+--------+---------+----------+
|       id|latitude|longitude|population|
+---------+--------+---------+----------+
|Araripina|  7.5766|  40.4976|     84418|
|  Caruaru|  8.2760|  35.9819|    277982|
|  Carpina|  7.8450|  35.2437|     81054|
| Ouricuri|  7.8809|  40.0810|     69459|
|  Surubim|  7.8543|  35.7630|     64520|
|   Recife|  8.0522|  34.9286|   1555000|
+---------+--------+---------+----------+



In [17]:
g2.edges.show()

+-------+--------+------------+----+
|    src|     dst|relationship|cost|
+-------+--------+------------+----+
| Recife| Surubim|       EROAD| 118|
|Surubim| Caruaru|       EROAD|  67|
|Caruaru|Ouricuri|       EROAD| 492|
+-------+--------+------------+----+



**Filter 2: g3**

In [18]:
g3 = g.filterEdges("cost > 500").filterVertices("latitude > 7 and population > 100000")

In [19]:
g3.vertices.show()

+-------+--------+---------+----------+
|     id|latitude|longitude|population|
+-------+--------+---------+----------+
|Caruaru|  8.2760|  35.9819|    277982|
| Recife|  8.0522|  34.9286|   1555000|
+-------+--------+---------+----------+



In [20]:
g3.edges.show()

+------+-------+------------+----+
|   src|    dst|relationship|cost|
+------+-------+------------+----+
|Recife|Caruaru|       EROAD| 510|
+------+-------+------------+----+



### Exercise 6

### 1. Determine o nó central através do grau.

In [21]:
total_degree = g.degrees
in_degree    = g.inDegrees
out_degree   = g.outDegrees

In [22]:
(total_degree
    .join(in_degree, "id", how="left")
    .join(out_degree, "id", how="left")
    .fillna(0)
    .sort("inDegree", ascending=False)
    .show())

+---------+------+--------+---------+
|       id|degree|inDegree|outDegree|
+---------+------+--------+---------+
| Ouricuri|     8|       5|        3|
|Salgueiro|     8|       5|        3|
| Igarassu|     7|       4|        3|
|Petrolina|     6|       3|        3|
|  Caruaru|     6|       3|        3|
|  Surubim|     6|       3|        3|
|   Recife|     5|       2|        3|
|Araripina|     5|       2|        3|
|  Cabrobo|     5|       2|        3|
|  Carpina|     4|       1|        3|
+---------+------+--------+---------+



Por grau, os nós centrais são: Ouricuri e Salgueiro.

### 2. Determine o nó central por centralidade utilizando a distância de Wasserman e a distância harmônica (consultar material).

**Distância de Wasserman**

**Distancia Harmônica**

### 3. Determine o nó central por intermediação.